# testAPI_propane

Created by Davy Yue 2017-06-22


### Imports

In [ ]:
import itertools
import string
import os

import numpy as np

from msibi import MSIBI, State, Pair, mie
import mdtraj as md

## PROPANE - edition (code to be expanded)

Where the real magic happens

In [ ]:
t = md.load('traj_unwrapped.dcd', top='start_aa.hoomdxml')

# print info for debugging
# print(t)
# print("Number of Chains: {0}".format(t.n_chains))
# print("Topology: {0}".format(t.top))

### Mapping and application

Keys being CG bead indices and values being a list of atom indices corresponding to each CG bead

e.g., {prop0: [0, 1, 2], prop1: [3, 4, 5], prop2: [6, 7, 8], …}

Construct for entire system

In [ ]:
cg_idx = 0
start_idx = 0
propane_map = {0: [0, 1, 2]}
for n in range(n_propane):
    for bead, atoms in propane_map.items():
        System_mapping[cg_idx] = [x + start_idx for x in atoms]

With mapping for whole system, apply to all atom trajectory

In [ ]:
cg_xyz = np.empty((traj.n_frames, traj.n_atoms/3, 3))
for cg_bead, aa_indices in system_mapping.items():
    cg_xyz[:, cg_bead, :] = center_of_mass(aa_indices)

### Traj & Obj

Create new Trajectory object & CG Topology object - save resultant trajectory file

In [ ]:
cg_top = md.Topology()
for cg_bead in system_mapping.keys():
    cg_top.add_atom(...) #figure out content of parameters
    

cg_traj = md.Trajectory(cg_xyz, cg_top, # other arguments)
cg_traj.save(cg-traj.dcd)

### Calculate RDF and save

In [ ]:
md.compute_rdf(cg_traj, pairs) #refer to compute_rdf documentation
#add code to save outputted file